In [1]:
import pandas as pd
import numpy as np
import json
import re
import requests
from urllib.parse import urlparse


In [2]:
# train_df = pd.read_csv('../data/Constraint_Train.csv')
# val_df = pd.read_csv('../data/Constraint_Val.csv')
test_df = pd.read_csv('../data/english_test_with_labels.csv')
test_df.head()

,id,tweet,label
0,1,Our daily update is published. States reported...,real
1,2,Alfalfa is the only cure for COVID-19.,fake
2,3,President Trump Asked What He Would Do If He W...,fake
3,4,States reported 630 deaths. We are still seein...,real
4,5,This is the sixth time a global health emergen...,real


In [3]:
# train_feature_df = pd.read_csv('../Boosting/Boosting Data/Train_v3.csv')
# val_feature_df = pd.read_csv('../Boosting/Boosting Data/Validation_v3.csv')
# test_feature_df = pd.read_csv('../Boosting/Boosting Data/Test_v3.csv')
test_feature_df = pd.read_csv('../Boosting/Boosting Data/test.csv')
test_feature_df.head()

,id,model1_fake,model1_real,model2_fake,model2_real
0,1,0.025615,0.974385,0.000000,1.000000
1,2,0.944206,0.055794,0.859564,0.140436
2,3,0.997134,0.002866,1.000000,0.000000
3,4,0.113006,0.886994,0.000000,1.000000
4,5,0.346905,0.653095,0.604537,0.395463


In [4]:
def get_df(df, feature_df, mode=''):
  if mode in ['train', 'val']:
    df = df.merge(feature_df, on=['id'], how='left').drop('Label', 1)
  else:
    df = df.merge(feature_df, on=['id'], how='left')
  df = df.iloc[:, :-2]
  return df

In [5]:
# train_df = get_df(train_df, train_feature_df, 'train')
# val_df = get_df(val_df, val_feature_df, 'val')
test_df = get_df(test_df, test_feature_df, 'test')


In [6]:
username_dic = json.load(open("Statistical meta/train_prob_vectors.json"))
domain_dic = json.load(open("Statistical meta/train_prob_vectors_domain.json"))

In [7]:
list(username_dic.items())[0]

('MoHFW_INDIA',
 {'fake_probability': 0.037037037037037035,
  'real_probability': 0.9629629629629629,
  'total_mentions': 162})

In [8]:
def username_feature(row, index):
  tweet = row["tweet"]
  lis = re.findall("@([a-z0-9_]+)", tweet, re.I)
  val=0
  if len(lis)>0:
    for item in lis:
      try:
        val+=username_dic[item][index]
      except KeyError:
        val+=0
    val/=len(lis)
  
  return val


def domain_feature(row, domain_dic, index):
  tweet=row["tweet"]
  try:
    shorturl = re.search("(?P<url>https?://[^\s]+)", tweet).group("url")
    r = requests.get(shorturl, timeout=180)
    expanded_url = r.url 
    # temp_domain = expanded_url.split('/')[2]
    domain = urlparse(expanded_url).netloc
    return domain_dic[domain][index]
  except Exception as e:
    domain=''
    return 0.0



In [9]:
# train_df["username_real"] = train_df.apply(lambda x: username_feature(x, "real_probability"), 1)
# train_df["username_fake"] = train_df.apply(lambda x: username_feature(x, "fake_probability"), 1)

# train_df["domain_real"] = train_df.apply(lambda x: domain_feature(x, domain_dic, "real_probability"), 1)
# train_df["domain_fake"] = train_df.apply(lambda x: domain_feature(x, domain_dic, "fake_probability"), 1)


In [10]:
# val_df["username_real"] = val_df.apply(lambda x: username_feature(x, "real_probability"), 1)
# val_df["username_fake"] = val_df.apply(lambda x: username_feature(x, "fake_probability"), 1)

# val_df["domain_real"] = val_df.apply(lambda x: domain_feature(x, domain_dic, "real_probability"), 1)
# val_df["domain_fake"] = val_df.apply(lambda x: domain_feature(x, domain_dic, "fake_probability"), 1)


In [ ]:
test_df["username_real"] = test_df.apply(lambda x: username_feature(x, "real_probability"), 1)
test_df["username_fake"] = test_df.apply(lambda x: username_feature(x, "fake_probability"), 1)

test_df["domain_real"] = test_df.apply(lambda x: domain_feature(x, domain_dic, "real_probability"), 1)
test_df["domain_fake"] = test_df.apply(lambda x: domain_feature(x, domain_dic, "fake_probability"), 1)


In [ ]:
# domain_dic

In [ ]:
# pd.options.display.max_rows=None
test_df[test_df["domain_fake"]!=0.0]

In [ ]:
test_df.to_csv('Post Processing/data/test_with_postproc_features.csv', index=False)